In [19]:
from transformers import pipeline
import soundfile as sf
#from transformers import AutoProcessor, MusicgenMelodyForConditionalGeneration, MusicgenForConditionalGeneration
from diffusers import StableAudioPipeline
import numpy as np
import sys, torch
from tqdm import tqdm as notebook_tqdm

MAX_TOKENS: int = 1503 # 30 seconds of tokens

In [20]:
pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda:1")

# define the prompts
prompt = "a calm creek with a gentle stream, birds chirping, and a soft breeze"
negative_prompt = "Low quality."

# set the seed for generator
generator = torch.Generator("cuda:1").manual_seed(0)

num_waveforms: int = 1

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [18]:
# run the generation
audio = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=200,
    audio_end_in_s=10.0,
    num_waveforms_per_prompt=num_waveforms,
    generator=generator,
).audios

print(audio.shape)

for i in range(num_waveforms):
    output = audio.squeeze(0).T.float().cpu().numpy()
    sf.write(f"sounds/test.wav", output, pipe.vae.sampling_rate)

  0%|          | 0/200 [00:00<?, ?it/s]

/home/evanmm3/ms-research/venv/lib/python3.10/site-packages/torchsde/_brownian/brownian_interval.py:608: UserWarning: Should have tb<=t1 but got tb=500.00006103515625 and t1=500.0.
  warnings.warn(f"Should have {tb_name}<=t1 but got {tb_name}={tb} and t1={self._end}.")


KeyboardInterrupt: 

In [40]:
a, sr = sf.read('sounds/sound_0_1.wav', dtype='float32')
print(a.shape, sr)

(441000, 2) 44100


## Embeddings + textual inversion

In [21]:
import torch
from diffusers import StableAudioPipeline
import laion_clap
import soundfile as sf
import accelerate
import torchaudio

In [22]:
# 1. Initialize the CLAP_Module without loading
model = laion_clap.CLAP_Module(enable_fusion=False, amodel="HTSAT-tiny").to("cuda:1")

model.load_ckpt()

# # 2. Load the raw checkpoint
# ckpt = torch.load("630k-audioset-best.pt", map_location="cuda:1")

# # 3. Remove the rogue key(s)
# ckpt_clean = {k: v for k, v in ckpt.items()
#               if "text_branch.embeddings.position_ids" not in k}

# # 4. Manually load into the underlying PyTorch model
# model.model.load_state_dict(ckpt_clean, strict=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [ ]:
projector = torch.nn.Linear(512, 768).to("cuda:1")

# -----------------------------------------------
# Audio embedding

audio_file = [
    'sounds/sound_0_1.wav',
]
audio_embed = projector(model.get_audio_embedding_from_filelist(x = audio_file, use_tensor=True)).unsqueeze(0).to(torch.float16)
print('Audio -> ', audio_embed.shape)

# -----------------------------------------------
# text embedding

text_data = ["high quality"]
text_embed = projector(model.get_text_embedding(text_data, use_tensor=True)).unsqueeze(0).to(torch.float16)
print('Text -> ', text_embed.shape)

print(text_embed.dtype, audio_embed.dtype)


Audio ->  torch.Size([1, 1, 768])
Text ->  torch.Size([1, 1, 768])
torch.float16 torch.float16


In [ ]:
from diffusers.models.transformers.stable_audio_transformer import StableAudioDiTModel
import sys
sys.path.append('./src/')
from diffusers.pipelines.stable_audio import StableAudioPipeline

transformer = StableAudioDiTModel.from_pretrained(
        "stabilityai/stable-audio-open-1.0",
        subfolder="transformer",
        torch_dtype=torch.float16
    )

pipe = StableAudioPipeline.from_pretrained(
        "stabilityai/stable-audio-open-1.0",
        torch_dtype=torch.float16,
        transformer=transformer,
    )

pipe = pipe.to("cuda:1")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/home/evanmm3/ms-research/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [43]:
_ = pipe.to("cuda:1")

text_embed = text_embed.to("cuda:1")
audio_embed = audio_embed.to("cuda:1")
print(text_embed.shape, audio_embed.shape)

# use __call__ to generate
audio = pipe.__call__(
    num_inference_steps=200,
    audio_end_in_s=10.0,
    num_waveforms_per_prompt=num_waveforms,
    generator=generator,
    prompt_embeds=audio_embed + text_embed, # add the audio and text embeddings 
).audios

# out.audios is a list of tensors; take the first one
print(audio.shape)
generated = audio.squeeze(0).T.float().cpu().numpy()

# 5. Save it
sf.write("sounds/complete.wav", generated, samplerate=pipe.vae.sampling_rate)

torch.Size([1, 1, 768]) torch.Size([1, 1, 768])


  0%|          | 0/200 [00:00<?, ?it/s]

torch.Size([1, 2, 441000])
